In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import transformers
import torch
import torchvision
import torchaudio
import numpy as np
import pandas as pd
import moviepy
import librosa
import glob
import moviepy.editor as mp
from torch.utils.data import TensorDataset, DataLoader, random_split
import librosa.display
import matplotlib.pyplot as plt
import tarfile
import matplotlib
import tqdm
import PIL
import cv2 as cv
import os
from os import walk, listdir
import shutil
import subprocess
import shlex
from tqdm import tqdm
import pickle
from transformers import AutoTokenizer, BertForTokenClassification, BertForSequenceClassification,BertPreTrainedModel, BertModel, BertTokenizer, AutoModel
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from scipy.io.wavfile import read
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from transformers import ViTFeatureExtractor, ViTModel
import requests
from sys import stderr, stdout
from torchvggish import vggish, vggish_input
import random
import re

EXTRACTING AUDIO FROM VIDEOS

In [ ]:
VIDEO_DIR = '/path/to/folder/containing/videos/'
OUTPUT_DIR = '/path/to/folder/for/storing/audio files'

CATEGORY = "Explicit Hate Videos" #change it to Implicit Hate Videos, Non Hate Videos as required

BW = 160
OC = 1
OF = 16000

In [ ]:
# Excel file containing the name and ids of videos
df1 = pd.read_excel(f"{VIDEO_DIR}/{CATEGORY}.xlsx")

In [ ]:
def has_audio_stream(video_path):
    """Check if the video has an audio stream."""
    command = f"ffmpeg -i {shlex.quote(video_path)}"
    result = subprocess.run(command, shell=True, stderr=subprocess.PIPE, text=True)
    return "Audio:" in result.stderr

def vToA():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    cnt = 0

    for video_id in df1["Video_ID"]:
        video = video_id + '.mp4'
        video_path = os.path.join(VIDEO_DIR, video)
        output_audio_path = os.path.join(OUTPUT_DIR, video_id + '.wav')

        print(f"Processing: {video_path}")
        
        if not os.path.isfile(video_path):
            print(f"Video file not found: {video_path}. Skipping.")
            continue
        
        if not has_audio_stream(video_path):
            print(f"No audio stream in {video}. Skipping.")
            continue

        with open(os.devnull, "w") as ffmpeg_log:
            try:
                command = (
                    f"ffmpeg -i {shlex.quote(video_path)} -ab {BW}k -ac {OC} "
                    f"-ar {OF} -vn {shlex.quote(output_audio_path)}"
                )
                print(f"Executing command: {command}")
                ret = subprocess.call(command, shell=True, stdout=ffmpeg_log, stderr=ffmpeg_log)

                if ret != 0:
                    print(f"Error processing {video}. Skipping.")
                    continue

                cnt += 1
                print(f"Processed {cnt}: {video}")

            except Exception as e:
                print(f"An error occurred with {video}: {e}")

    print(f"Conversion complete. Successful conversions: {cnt}")

vToA()